In [ ]:
#Idea explained:
#https://www.kaggle.com/competitions/playground-series-s4e7/discussion/516462#2902952

#Notebook detailing:
#https://www.kaggle.com/code/yeoyunsianggeremie/85-7-simple-probabilistic-approach-no-ml-model/notebook

In [30]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

In [54]:
train = pd.read_csv("..\\data\\kaggle\\train.csv", nrows=500000).drop(columns=["id"])
print(train.shape)
train[:2]

(500000, 11)


,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1


In [52]:
test = pd.read_csv("..\\data\\kaggle\\test.csv").drop(columns=["id"])
print(test.shape)
test[:2]

(7669866, 10)


,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228
1,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123


In [55]:
probabilities = {}

#para cada feature:
for col in train.columns.drop('Response'):
    #para cada linha, e variações da feature, calcula a média da var Response para a variação da feature: obterá a probabilidade daquela variação de feature ser 1 (probabilidade de aceite).
    #print(col)
    prob = train.groupby(col)['Response'].mean()
    #print(prob)
    #print('\n')

    #inclui as probabilidades de todas as variações das features em um dicionário
    probabilities[col] = prob
    #print(probabilities)
    #print('\n')

    #em cada linha e coluna, substitui o valor original da feature pela probabilidade de aceite.
    train[col] = train[col].map(probabilities[col])
    #print(train[col])

#aplica para teste também
for col in test.columns:
    test[col] = test[col].map(probabilities[col])

In [56]:
#Resultado: ao invés do valor da feature, tenho a probabilidade de aceite para cada feature
train[:5]

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0.139512,0.031060,0.122411,0.116638,0.227004,0.176544,0.239101,0.333333,0.192542,0.085525,0
1,0.139512,0.221802,0.122411,0.190302,0.227004,0.296865,0.239101,0.125000,0.204525,0.171813,1
2,0.101905,0.031516,0.122411,0.090676,0.000670,0.041455,0.004305,0.200000,0.025831,0.077292,0
3,0.101905,0.218463,0.122411,0.103472,0.227004,0.176544,0.239101,0.127539,0.201420,0.107924,0
4,0.101905,0.226618,0.122411,0.066794,0.000670,0.176544,0.004305,0.100000,0.025831,0.110911,0


In [57]:
test[:5]

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,0.101905,0.023534,0.122411,0.084930,0.227004,0.041455,0.004305,0.127539,0.020637,0.058355
1,0.139512,0.219546,0.122411,0.190302,0.227004,0.176544,0.239101,0.269231,0.192542,0.211576
2,0.139512,0.219546,0.122411,0.099523,0.227004,0.176544,0.239101,0.127539,0.204525,0.120985
3,0.101905,0.032822,0.122411,0.084930,0.000670,0.041455,0.004305,0.000000,0.025831,0.109807
4,0.139512,0.187906,0.122411,0.161120,0.227004,0.176544,0.004305,0.350000,0.192542,0.266667


In [58]:
#NaNs, se poucas linhas em teste, pois não houve amostra em teste, com muitas linhas terá.

In [59]:
#salva a resposta do treino em nova var:
ground_truth = train["Response"].copy()
#dropa a resposta:
train.drop(columns = ["Response"], inplace=True)

In [60]:
#cria nova col, somando as probabilidades de todas as features da linha, assumindo que seja quanto ela contribui para um aceite de seguro. 
    #Prediction vai ficar > 1 as vezes, mas como usaremos ROC_AUC, não tem problema
train["Prediction"] = train.sum(axis = 1)

In [61]:
train[:3]

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Prediction
0,0.139512,0.031060,0.122411,0.116638,0.227004,0.176544,0.239101,0.333333,0.192542,0.085525,1.663670
1,0.139512,0.221802,0.122411,0.190302,0.227004,0.296865,0.239101,0.125000,0.204525,0.171813,1.938334
2,0.101905,0.031516,0.122411,0.090676,0.000670,0.041455,0.004305,0.200000,0.025831,0.077292,0.696060


In [62]:
#concatenar var resposta:
train["Response"] = ground_truth
train[:3]

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Prediction,Response
0,0.139512,0.031060,0.122411,0.116638,0.227004,0.176544,0.239101,0.333333,0.192542,0.085525,1.663670,0
1,0.139512,0.221802,0.122411,0.190302,0.227004,0.296865,0.239101,0.125000,0.204525,0.171813,1.938334,1
2,0.101905,0.031516,0.122411,0.090676,0.000670,0.041455,0.004305,0.200000,0.025831,0.077292,0.696060,0


In [63]:
#Avalia com ROC_AUC
print(roc_auc_score(train["Response"], train["Prediction"]))

0.882018944308113


In [64]:
#submete as probabilidades como resposta:
test["Response"] = test.sum(axis=1)
test[:3]

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0.101905,0.023534,0.122411,0.084930,0.227004,0.041455,0.004305,0.127539,0.020637,0.058355,0.812075
1,0.139512,0.219546,0.122411,0.190302,0.227004,0.176544,0.239101,0.269231,0.192542,0.211576,1.987768
2,0.139512,0.219546,0.122411,0.099523,0.227004,0.176544,0.239101,0.127539,0.204525,0.120985,1.676690


In [67]:
#substituir as respostas do sample_submission:
submission = pd.read_csv("..\\data\\kaggle\\sample_submission.csv")
submission[:3]

,id,Response
0,11504798,0.5
1,11504799,0.5
2,11504800,0.5


In [68]:
submission["Response"] = test["Response"]
submission

,id,Response
0,11504798,0.812075
1,11504799,1.987768
2,11504800,1.676690
3,11504801,0.524135
4,11504802,1.828009
...,...,...
7669861,19174659,1.781949
7669862,19174660,0.566809
7669863,19174661,1.067849
7669864,19174662,1.559655


In [71]:
# Export to submit on Kaggle
submission.to_csv('..\\data\\v3\\submission_v3.csv', index=False)